In [16]:
import sys
from pyspark.sql import SparkSession, functions, types

In [2]:
# boilerplate code
spark = SparkSession.builder.appName('weather ETL').getOrCreate()
spark.sparkContext.setLogLevel('WARN')

assert sys.version_info >= (3, 5) # make sure we have Python 3.5+
assert spark.version >= '2.3' # make sure we have Spark 2.3+

In [3]:
# define the schema for when the data is imported
observation_schema = types.StructType([
    types.StructField('station', types.StringType()),
    types.StructField('date', types.StringType()),
    types.StructField('observation', types.StringType()),
    types.StructField('value', types.IntegerType()),
    types.StructField('mflag', types.StringType()),
    types.StructField('qflag', types.StringType()),
    types.StructField('sflag', types.StringType()),
    types.StructField('obstime', types.StringType()),
])

In [10]:
# main function 
def main():
    in_directory = 'weather-1/'
    out_directory = 'output/'

    weather = spark.read.csv(in_directory, schema=observation_schema)

    # TODO: finish here.
    # DONE
    cleaned_data = weather.filter(weather.qflag.isNull())\
        .filter(weather.station.startswith('CA'))\
        .filter(weather.observation.startswith('TMAX'))\
        .select(weather['station'], weather['date'], (weather.value/10).alias('tmax'))
    cleaned_data.show()

    cleaned_data.write.json(out_directory, compression='gzip', mode='overwrite')
    return cleaned_data

In [11]:
if __name__ == '__main__':
    main()

+-----------+--------+----+
|    station|    date|tmax|
+-----------+--------+----+
|CA001157630|20161203| 4.5|
|CA004015322|20161203| 2.1|
|CA003010162|20161203| 2.5|
|CA001085836|20161203| 2.2|
|CA006135583|20161203| 5.0|
|CA007093714|20161203|-9.1|
|CA007018563|20161203| 1.6|
|CA001184791|20161203| 2.4|
|CA002400573|20161203|-7.2|
|CA006016529|20161203|-5.7|
|CA00615EMR7|20161203| 4.0|
|CA001101158|20161203| 6.5|
|CA002402353|20161203|-7.6|
|CA001077499|20161203| 3.7|
|CA004010879|20161203|-1.2|
|CA008403690|20161203| 1.5|
|CA004016322|20161203|-0.5|
|CA001173210|20161203| 1.0|
|CA007061288|20161203|-4.2|
|CA003032550|20161203| 8.4|
+-----------+--------+----+
only showing top 20 rows



In [12]:
data = main()

+-----------+--------+----+
|    station|    date|tmax|
+-----------+--------+----+
|CA001157630|20161203| 4.5|
|CA004015322|20161203| 2.1|
|CA003010162|20161203| 2.5|
|CA001085836|20161203| 2.2|
|CA006135583|20161203| 5.0|
|CA007093714|20161203|-9.1|
|CA007018563|20161203| 1.6|
|CA001184791|20161203| 2.4|
|CA002400573|20161203|-7.2|
|CA006016529|20161203|-5.7|
|CA00615EMR7|20161203| 4.0|
|CA001101158|20161203| 6.5|
|CA002402353|20161203|-7.6|
|CA001077499|20161203| 3.7|
|CA004010879|20161203|-1.2|
|CA008403690|20161203| 1.5|
|CA004016322|20161203|-0.5|
|CA001173210|20161203| 1.0|
|CA007061288|20161203|-4.2|
|CA003032550|20161203| 8.4|
+-----------+--------+----+
only showing top 20 rows



In [25]:
data.cache()
result = data.groupBy('date').count().sort('date')

In [33]:
data.groupBy('date').agg({'tmax': 'count'}).show()

+--------+-----------+
|    date|count(tmax)|
+--------+-----------+
|20161203|         84|
+--------+-----------+

